# Tutorial: Use regression and classification to link data to outcomes

## Step 1: Regression

The first part of this notebook will simulate some data with known continuous relationships and then use supervised machine learning to recover those relationships. Let's start by linking a few variables to a target variable. Let's say we have a dataset with four features: `x1`, `x2`, `x3`, and `notassociated`. We want to predict a target variable `y` based on these features. The relationships are as follows:

* `y = 2*x1 + 3*x2 + 5*x3 + 2.5*x1**2 + 0*notassociated + e`

Let's go ahead and simulate 1000 samples of this data. We'll also split the data into training and testing sets so that we can evaluate the performance of our model.

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(24601)

n_samples = 1000
x1 = np.random.randn(n_samples)
x2 = np.random.randn(n_samples)
x3 = np.random.randn(n_samples)
notassociated = np.random.randn(n_samples)
e = np.random.normal(0, 1, n_samples)
y = 2*x1 + 3*x2 + 5*x3 + 2.5*x1**2 + 0*notassociated + e
data = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'x1sq': x1 ** 2,
    'notassociated': notassociated,
    'y': y
})
# Split into training and testing sets
train_data = data[:800]
test_data = data[800:]
print(data.head())

OK, we have the data. Now, let's visualize the relationships between the features and the target variable. We'll create scatter plots to see how each feature relates to `y`.

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(data['x1'], data['y'])
plt.xlabel('x1')
plt.ylabel('y')
plt.show()
plt.scatter(data['x2'], data['y'])
plt.xlabel('x2')
plt.ylabel('y')
plt.show()
plt.scatter(data['x3'], data['y'])
plt.xlabel('x3')
plt.ylabel('y')
plt.show()
plt.scatter(data['notassociated'], data['y'])
plt.xlabel('notassociated')
plt.ylabel('y')
plt.show()


So far, so good. Now, let's use the typical statistical approach to recover the relationships. We'll use OLS regression to estimate the coefficients for each feature. We'll use the `pingouin` library for this purpose. However, you can use any software you prefer (Stata, SPSS, R, etc.).



In [ ]:
import pingouin as pg

lm = pg.linear_regression(data[['x1', 'x2', 'x3', 'x1sq', 'notassociated']], data['y'])
print(lm)


That looks good. The coefficients are close to the true values we used to generate the data. Now, let's see if we can recover these relationships using machine learning. We'll use a simple linear regression model from `scikit-learn` to predict `y` based on `x1`, `x2`, and `x3`.

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(train_data[['x1', 'x2', 'x3', 'x1sq', 'notassociated']], train_data['y'])
print(f"Intercept: {lm.intercept_}")
print(f"Coefficients: {lm.coef_}")


The results are similar to what we got with OLS regression. The coefficients are close to the true values, and the model seems to be performing well. This is a good sign that our machine learning model is able to recover the relationships between the features and the target variable. Notice, however, that there are no p-values. This is a limitation of machine learning models. They don't provide p-values, which are useful for hypothesis testing and understanding the statistical significance of the relationships.

Rather than using p-values, supervised machine learning generally looks at the model's performance on a `test` set of data that was not used to train the model to show that the model fits. Let's take a look at that fit.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
predictions = lm.predict(test_data[['x1', 'x2', 'x3', 'x1sq', 'notassociated']])
print(f"R-squared: {r2_score(test_data['y'], predictions)}")
print(f"MSE: {mean_squared_error(test_data['y'], predictions)}")
plt.scatter(test_data['y'], predictions)
plt.xlabel('y')
plt.ylabel('predictions')
plt.show()


Notice that we calculated the fit based on the `test_data`? This is important because it allows us to evaluate the model's performance on data that it has not seen before. This is a good way to assess the model's generalization ability.

That looks pretty accurate! Let's take a look at the residuals to see if there are any patterns we should be concerned about.

In [ ]:
residuals = test_data['y'] - predictions
plt.scatter(predictions, residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()


If you wans something slightly more sophisticated, you can merge the two together and show the fit line AND the residuals.

In [ ]:
plt.scatter(test_data['y'], predictions, label='Predictions', alpha=0.6)
plt.plot(test_data['y'], test_data['y'], color='red', linestyle='--', label='Perfect Fit')
plt.errorbar(test_data['y'], predictions, yerr=abs(residuals), fmt='o', alpha=0.3, color='gray')
plt.xlabel('Actual y')
plt.ylabel('Predicted y')
plt.title('Actual vs Predicted with Error Bars')
plt.legend()
plt.show()

Looks pretty good to me! The residuals are randomly distributed around zero, which is a good sign that our model is capturing the relationships between the features and the target variable. 

One of the limitations of OLS/linear regression is that it assumes a linear relationship between the features and the target variable. However, in many real-world scenarios, the relationships are not linear. In such cases, we need to use more complex models that can capture these non-linear relationships.

Now, let's let's try a more complex model and we'll leave out the `x1sq` variable. We'll use a random forest regressor from `scikit-learn`. Random forests are a type of ensemble learning method that can capture complex relationships between features and the target variable.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)
features = ['x1', 'x2', 'x3', 'notassociated']
rf.fit(train_data[features], train_data['y'])
print(f"Random Forest has been fitted.\n\nFeature importances:")
for importance, feature in zip(rf.feature_importances_, features):
    print(f"{feature}: {importance}")
predictions = rf.predict(test_data[features])

Because a random forest estimator does not assume a linear relationship between the predictors and y, there are no 'regression coefficients' to report. However, we can still see the importance of the variables here

In these data it looks like:
* x3 is the most important. This seems consistent with the true relationship we used to generate the data, with a coefficient of 5.
* x1 is the second most important. While the direct effect of x1 on y is 2, the squared term also contributes to the relationship. So this makes sense as well
* x2 comes in third. This is also consistent with the true relationship we used to generate the data, with a coefficient of 3.
* notassociated is the least important by far. This is also consistent with the true relationship we used to generate the data, with a coefficient of 0.

Let's look at the fit of the model again, like we did with linear regression.

In [ ]:
print(f"R-squared: {r2_score(test_data['y'], predictions)}")
print(f"MSE: {mean_squared_error(test_data['y'], predictions)}")

plt.scatter(test_data['y'], predictions, label='Predictions', alpha=0.6)
plt.plot(test_data['y'], test_data['y'], color='red', linestyle='--', label='Perfect Fit')
plt.errorbar(test_data['y'], predictions, yerr=abs(residuals), fmt='o', alpha=0.3, color='gray')
plt.xlabel('Actual y')
plt.ylabel('Predicted y')
plt.title('Actual vs Predicted with Error Bars')
plt.legend()
plt.show()

Well, that certainly doesn't look as good as the OLS regression fit. But wait, we expected the Random Forest regressor to 'figure out' the non-linear relationship between x1 and y, right? Let's see what would've happened to OLS if we had expected the same of it!

In [ ]:
lm = LinearRegression()
lm.fit(train_data[features], train_data['y'])
print(f"Intercept: {lm.intercept_}")
print(f"Coefficients: {lm.coef_}")
predictions = lm.predict(test_data[features])
print(f"R-squared: {r2_score(test_data['y'], predictions)}")
print(f"MSE: {mean_squared_error(test_data['y'], predictions)}")
plt.scatter(test_data['y'], predictions, label='Predictions', alpha=0.6)
plt.plot(test_data['y'], test_data['y'], color='red', linestyle='--', label='Perfect Fit')
plt.errorbar(test_data['y'], predictions, yerr=abs(residuals), fmt='o', alpha=0.3, color='gray')
plt.xlabel('Actual y')
plt.ylabel('Predicted y')
plt.title('Actual vs Predicted with Error Bars')
plt.legend()
plt.show()

Yuck! While the Random Forest regressor did not capture the non-linear relationship between x1 and y as well as OLS did when we had explicitly modeled it, OLS did a terrible job of capturing the non-linear relationship between x1 and y unless we explicitly modeled it. This is a good example of how machine learning can sometimes outperform traditional statistical methods when it comes to capturing complex relationships between variables.

## Step 2: Understanding the relationships without regression coefficients

As we saw above, we know that x1, x2, and x3 are important in predicting y. But we don't know how they are related to y. We can use a technique called SHAP (SHapley Additive exPlanations) values to understand the relationships between the features and the target variable. SHAP values are a way to explain the output of any machine learning model by assigning each feature an importance value for a particular prediction. Let's use the `shap` library to calculate the SHAP values for our random forest model. We'll create a summary plot to visualize the relationships between the features and the target variable.

First, let's get our random forest model back.

In [ ]:
rf = RandomForestRegressor(random_state=24601)
features = ['x1', 'x2', 'x3', 'notassociated']
rf.fit(train_data[features], train_data['y'])
print(f"Random Forest has been fitted")

OK, now let's calculate the SHAP values and create a summary plot.

In [ ]:
import shap

explainer = shap.Explainer(rf, test_data[['x1', 'x2', 'x3', 'notassociated']])
shap_values = explainer(test_data[['x1', 'x2', 'x3', 'notassociated']])
shap.summary_plot(shap_values, test_data[['x1', 'x2', 'x3', 'notassociated']])

Let's make sense of this plot
* The x-axis represents the SHAP value for each feature. A positive SHAP value indicates that the feature is pushing the prediction higher, while a negative SHAP value indicates that the feature is pushing the prediction lower.
* The y-axis represents the features. Each point on the plot represents a SHAP value for a particular feature and a particular prediction.
* The color of the points represents the value of the feature. For example, if the feature value is high, the point will be red, and if the feature value is low, the point will be blue.

Here we can see that:
* x3 has a big effect, higher values of x3 (more red) push the prediction higher (more positive SHAP values), while lower values of x3 (more blue) push the prediction lower (more negative SHAP values).
* x1 has a non-linear effect. Both high and low values of x1 (more red and more blue) push the prediction higher (more positive SHAP values), while values around 0 (more purple) have a small effect (closer to zero). But we also see that high values of x1 (more red) stretch out further than the low values of x1 (more blue). Together, this is consistent with the true relationship we used to generate the data, where the effect of x1 on y is quadratic, but has a small positive linear effect as well.
* x2 has a linear effect. Higher values of x2 (more red) push the prediction higher (more positive SHAP values), while lower values of x2 (more blue) push the prediction lower (more negative SHAP values). However, the stretch of this plot being more constrained than x3 suggests that x2 has a smaller effect on y than x3. This again is consistent with the true relationship we used to generate the data, where the effect of x2 on y is linear, but the coefficient is smaller than that of x3.
* notassociated has no effect. The SHAP values are all close to zero, which is consistent with the true relationship we used to generate the data, where the coefficient for notassociated is 0.

So, no, we don't have p-values or coefficients... but we can still make sense of the relationships between the features and the target variable. And we were able to ascertain a curvilinear relationship between x1 and y that OLS missed. Neat!

We can also examine partial dependence plots to see the relationship between each feature and the target variable. Partial dependence plots show the average predicted response as a function of a feature, marginalizing over the values of all other features. Let's create partial dependence plots for each feature.

In [ ]:
from sklearn.inspection import PartialDependenceDisplay

fig, ax = plt.subplots(figsize=(10, 6))
PartialDependenceDisplay.from_estimator(rf, data[features], features, ax=ax)
plt.show()


Here too, we see the associated relationships more clearly and without presupposing linearity.

## Part 3: Hyperparameter Tuning and Model Selection

In the above sections, we have used default hyperparameters for our machine learning models. In statistical models, we often have very few hyperparameters to set other than the model type. For instance, in OLS regression, we might estimate a model with or without an intercept or with robust standard errors. However, in machine learning we often have many hyperparameters to tune that influences how the model learns from the data. For example, in a random forest model, we might tune the number of trees, the maximum depth of the trees, and the minimum samples required to split a node. I know these don't mean much to you, but the goal of this class is not to teach you the various machine learning algorithms and their hyperparameters, but rather to expose you to the models and what is possible with them, and let you explore on your own when you determine you will use one or more of these models.

Let's take a look at the hyperparameters for the random forest model we used above.

In [ ]:
#display hyperparameters

rf.get_params()

That's a lot of hyperparameters! And this is just one model. There are many other machine learning models with their own set of hyperparameters (many of which do not substantively overlap with this list). This is where the "art" of machine learning comes in. Researchers often spend a lot of time tuning hyperparameters to get the best performance.

Now wait, that sounds antithetical to science. If we are tuning hyperparameters to get the best performance, are we not overfitting to our data? This is a valid concern, and it is why we need to be careful when tuning hyperparameters. We need to:
* Use theory where possible to guide our choice of hyperparameters
* Evaluate the performance of our model on a `test` set of data that was not used to train the model
* Use cross-validation to get a more robust estimate of the model's performance

Cross-validation is a technique for evaluating the performance of a model by training it on different subsets of the data and testing it on the remaining data. For example, let's assume we have a dataset with 1000 samples. We first split this dataset into two parts: a training set with 800 samples and a test set with 200 samples. We set the test set aside for now. We then split the training set into 5 `folds` of 160 samples each. We then train the model on 4 folds (640 samples) and test it on the remaining fold (160 samples). We repeat this process 5 times, each time using a different fold as the test set. This gives us 5 estimates of the model's performance. We can then average these estimates to get a more robust estimate of the model's performance. If this is still unclear, I recommend you watch [this video](https://www.youtube.com/watch?v=kituDjzXwfE).

We also want to use a technique called `randomized search` to find the best hyperparameters for our model. Randomized search is a technique for systematically searching through a range of hyperparameters to find the best combination. For example, let's say we want to tune the number of trees in our random forest model. We might try values ranging from 10 to 200. We would then train and evaluate the model for each value and select the one that gives us the best performance.

Let's see how this works in practice. We'll use the `GridSearchCV` class from `scikit-learn` to perform grid search with cross-validation. We'll tune the number of trees and the maximum depth of the trees in our random forest model.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer, r2_score

param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

kfold = KFold(n_splits=5, shuffle=True, random_state=24601)
rf_kfoldgrid = RandomForestRegressor(random_state=24601)
scorer = make_scorer(r2_score)

random_search = RandomizedSearchCV(
    estimator=rf_kfoldgrid,
    param_distributions=param_dist,
    n_iter=100,
    scoring=scorer,
    cv=kfold,
    n_jobs=-1,
    verbose=1
)

X = train_data[features]
y = train_data['y']
random_search.fit(X, y)

print(f"Best Parameters: {random_search.best_params_}")
print(f"Best R-squared on training dataset: {random_search.best_score_}")
predictions = random_search.predict(test_data[features])
print(f"R-squared on test dataset: {r2_score(test_data['y'], predictions)}")


Let's compare this to the performance of the model with default hyperparameters.

In [ ]:
print("Max Depth:",rf.get_params()['max_depth'])
print("Min_samples_leaf:",rf.get_params()['min_samples_leaf'])
print("Min_samples_split:",rf.get_params()['min_samples_split'])
print("N_estimators:",rf.get_params()['n_estimators'])
predictions = rf.predict(test_data[features])
print(f"R-squared on test dataset: {r2_score(test_data['y'], predictions)}")

They're pretty close! However, the default parameters are not always going to perform as well as the tuned parameters. Our dataset was pretty clean and the relationships were not too complex, chances are that in a more complex dataset, the tuned parameters would outperform the default parameters.

# Step 4: Classification

In machine learning, we call the task of predicting a continuous variable `regression` and the task of predicting a categorical variable `classification`. In this section, we will simulate some data with known categorical relationships and then use supervised machine learning to recover those relationships. Let's start by linking a few variables to a target variable. Let's say we have a dataset with four features: `x1`, `x2`, `x3`, and `notassociated`. We want to predict a target variable `y` based on these features. The relationships are as follows:

* y = 1 if 2*x1 + 3*x2 + 5*x3 + 2.5*x1**2 + 0*notassociated + e > 0, otherwise it's zero

Like before, let's go ahead and simulate 1000 samples of this data. We'll also split the data into training and testing sets so that we can evaluate the performance of our model.

In [ ]:
np.random.seed(24601)

n_samples = 1000
x1 = np.random.normal(0, 1, n_samples)
x2 = np.random.normal(0, 1, n_samples)
x3 = np.random.normal(0, 1, n_samples)
notassociated = np.random.normal(0, 1, n_samples)
e = np.random.normal(0, 0.5, n_samples)
y = (2*x1 + 3*x2 + 5*x3 + 2.5*x1**2 + 0*notassociated + e > 0).astype(int)

data = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'x1sq': x1 ** 2,
    'notassociated': notassociated,
    'y': y
})

train_data = data[:800]
test_data = data[800:]
print(data.head())

Note that the y variable is categorical rather than continuous. Let's visualize the data to see how each feature relates to y.

In [ ]:
# visualize the data
import matplotlib.pyplot as plt
plt.scatter(data['x1'], data['y'])
plt.xlabel('x1')
plt.ylabel('y')
plt.show()
plt.scatter(data['x2'], data['y'])
plt.xlabel('x2')
plt.ylabel('y')
plt.show()
plt.scatter(data['x3'], data['y'])
plt.xlabel('x3')
plt.ylabel('y')
plt.show()
plt.scatter(data['notassociated'], data['y'])
plt.xlabel('notassociated')
plt.ylabel('y')
plt.show()


OK, not as easy to visualize as a continuous variable, but we can still see some relationships. Now, let's use the typical statistical approach to recover the relationships. We'll use logistic regression to estimate the coefficients for each feature. We'll use the `pingouin` library again.

In [ ]:
logistic = pg.logistic_regression(train_data[['x1', 'x2', 'x3', 'x1sq', 'notassociated']], train_data['y'])
print(logistic)


Like before, the significant variables are those we anticipated. Now, let's see if we can recover these relationships using machine learning. We'll use a simple logistic regression model from `scikit-learn` to predict y based on these variables.

In [ ]:
from sklearn.linear_model import LogisticRegression
logistic = LogisticRegression(random_state=24601, C=1e10, tol=1e-6)
logistic.fit(train_data[['x1', 'x2', 'x3', 'x1sq', 'notassociated']], train_data['y'])
print(f"Intercept: {logistic.intercept_}")
print(f"Coefficients: {logistic.coef_}")

Pretty similar again. Did you notice that I changed a couple of hyperparameters from their defaults (`C` and `tol`)? I did this to make the model replicate the results from the statistical model. As I mentioned above, ML models have hyperparameters that we tune that affect the results and this time the defaults didn't mirror pingouin's results.

Now, let's look at the fit of the model. We'll use a confusion matrix to see how well the model is able to predict the correct class.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
predictions = logistic.predict(test_data[['x1', 'x2', 'x3', 'x1sq', 'notassociated']])
print(f"Accuracy: {accuracy_score(test_data['y'], predictions)}")
print(f"Precision: {precision_score(test_data['y'], predictions)}")
print(f"Recall: {recall_score(test_data['y'], predictions)}")
print(f"F1 Score: {f1_score(test_data['y'], predictions)}")
print(f"MCC: {matthews_corrcoef(test_data['y'], predictions)}")
cm = confusion_matrix(test_data['y'], predictions)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()


Here, because we have a binary outcome, we have a host of metrics we can use to evaluate the model's performance. Most of them derive from the confusion matrix above.
* Accuracy: The proportion of correct predictions (both true positives and true negatives) out of all predictions. In this case, the model correctly predicted 97% of the cases.
* Precision: The proportion of true positives out of all positive predictions. In this case, the precision is 0.98, which means that 98% of positive predictions were correct.
* Recall: The proportion of true positives out of all actual positives. In this case, the recall is 0.97, which means that the model correctly identified 97% of the positive cases.
* F1 Score: The harmonic mean of precision and recall. It is a good measure of a model's accuracy when the class distribution is imbalanced. In this case, the F1 score is 0.97, which is very good.
* MCC: The Matthews correlation coefficient is a measure of the quality of binary classifications. It takes into account true positives, true negatives, false positives, and false negatives, making it a great indicator of how well a classifier works. In this case, the MCC is 0.92, which is very good.

There is also a ROC curve and AUC metric that we can use to evaluate the model's performance. The ROC curve is a plot of the true positive rate against the false positive rate at various threshold settings. The AUC (Area Under the Curve) is the area under the ROC curve and provides a single value to summarize the model's performance. An AUC of 1 indicates a perfect model, while an AUC of 0.5 indicates a model that is no better than random guessing. Let's take a look at the ROC curve and AUC for our model.

In [ ]:
#Plot ROC curve
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(test_data['y'], predictions)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, label='AUC = %0.2f' % roc_auc)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


Here the AUC is 0.96, which is very good. This indicates that the model is able to distinguish between the two classes with high accuracy. We also see that the blue curve (our model) is consistently above the orange curve (random guessing), which is a good sign.

As before though, we fed the model the squared term. Let's see what happens if we don't do that and let the model 'figure it out' for itself.

In [ ]:
features = ['x1', 'x2', 'x3', 'notassociated']
logistic = LogisticRegression(random_state=24601, C=1e10, tol=1e-6)
logistic.fit(train_data[features], train_data['y'])
print(f"Intercept: {logistic.intercept_}")
print(f"Coefficients: {logistic.coef_}")
predictions = logistic.predict(test_data[features])
print(f"MCC: {matthews_corrcoef(test_data['y'], predictions)}")
cm = confusion_matrix(test_data['y'], predictions)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()
fpr, tpr, _ = roc_curve(test_data['y'], predictions)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, label='AUC = %0.2f' % roc_auc)
plt.plot([0, 1], [0, 1], 'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


Uh oh... MCC went down to 0.67 and the AUC is down to 0.84.. We're not doing so hot anymore. Can we tune hyperparameters to improve our model?

In [ ]:
from scipy.stats import loguniform
param_dist = {
    'C': loguniform(1e-5, 1e5),
    'solver': ['lbfgs', 'liblinear', 'newton-cg', 'saga'],
    'penalty': ['l2'],
    'max_iter': [100, 500, 1000],
    'tol': loguniform(1e-6, 1e-2),
    'class_weight': [None, 'balanced']
}
kfold = KFold(n_splits=5, shuffle=True, random_state=24601)
logistic = LogisticRegression(random_state=24601)
mcc_scorer = make_scorer(matthews_corrcoef)
random_search = RandomizedSearchCV(
    estimator=logistic,
    param_distributions=param_dist,
    n_iter=100,
    scoring=mcc_scorer,
    cv=kfold,
    verbose=1,
    n_jobs=-1,
    random_state=24601
)
random_search.fit(train_data[features], train_data['y'])

print("Best Parameters:", random_search.best_params_)
print(f"Best MCC on training dataset: {random_search.best_score_}")
predictions = random_search.predict(test_data[features])
print(f"MCC on test dataset: {matthews_corrcoef(test_data['y'], predictions)}")

We're doing slightly better, MCC is up to 0.71. However, clearly, the model is still struggling. This is because logistic regression still assumes linearity, just in a different way than OLS (logit transformation). In other words, it cannot figure out the curvilinear relationship between x1 and y. Let's try a more complex model and we'll leave out the x1sq variable. We'll use a random forest classifier from `scikit-learn`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

param_dist = {
    'n_estimators': np.arange(50, 300, 50),
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

kfold = KFold(n_splits=5, shuffle=True, random_state=24601)
rf_kfoldgrid = RandomForestClassifier(random_state=24601)
scorer = make_scorer(matthews_corrcoef)
random_search = RandomizedSearchCV(
    estimator=rf_kfoldgrid,
    param_distributions=param_dist,
    n_iter=100,
    scoring=scorer,
    cv=kfold,
    n_jobs=-1,
    verbose=1
)
X = train_data[features]
y = train_data['y']
random_search.fit(X, y)
print(f"Best Parameters: {random_search.best_params_}")
print(f"Best MCC on training dataset: {random_search.best_score_}")
predictions = random_search.predict(test_data[features])
print(f"MCC on test dataset: {matthews_corrcoef(test_data['y'], predictions)}")


The MCC is up to 0.80. Much better! But now we don't have coefficients anymore again! Let's again use the partial dependence plots to understand the relationships between the features and the target variable.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
PartialDependenceDisplay.from_estimator(random_search.best_estimator_, data[features], features, ax=ax)
plt.show()

While we could stop here, let's also use Naive Bayes and see how it does because the readings this week talked about it.

In [ ]:
from sklearn.naive_bayes import GaussianNB

param_dist = {"var_smoothing": np.logspace(-12, -3, 100)}

kfold = KFold(n_splits=5, shuffle=True, random_state=24601)

nb_kfoldgrid = GaussianNB()
scorer = make_scorer(matthews_corrcoef)

random_search = RandomizedSearchCV(
    estimator=nb_kfoldgrid,
    param_distributions=param_dist,
    n_iter=100,
    scoring=scorer,
    cv=kfold,
    n_jobs=-1,
    verbose=1,
)
X = train_data[features]
y = train_data["y"]
random_search.fit(X, y)

print(f"Best Parameters: {random_search.best_params_}")
print(f"Best MCC on training dataset: {random_search.best_score_}")
predictions = random_search.predict(test_data[features])
print(f"MCC on test dataset: {matthews_corrcoef(test_data['y'], predictions)}")

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
PartialDependenceDisplay.from_estimator(random_search.best_estimator_, data[features], features, ax=ax)
plt.show()

The MCC for Naive Bayes was 0.84 - even better and the partial dependence plots are similar to the random forest classifier.

The choice of model depends on the specific problem and the characteristics of the data. In general, more complex models like random forests and Naive Bayes can capture more complex relationships between features and the target variable, but they also require more data to train effectively. Simpler models like logistic regression are easier to interpret and require less data, but they may not capture all the nuances of the data unless you explicitly model them. In practice, it's often a good idea to try several different models and compare their performance using cross-validation. This can help you find the best model for your specific problem and data.

Done...